### Test Create Topic

In [2]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --create --topic demo_1 --bootstrap-server localhost:9092 --partitions 2

Created topic demo_1.


In [5]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --bootstrap-server localhost:9092 --topic demo_1 --describe

Topic: demo_1	PartitionCount: 2	ReplicationFactor: 1	Configs: 
	Topic: demo_1	Partition: 0	Leader: 1	Replicas: 1	Isr: 1
	Topic: demo_1	Partition: 1	Leader: 1	Replicas: 1	Isr: 1


### Create 2nd Topic

In [6]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --create --topic demo_2 --bootstrap-server localhost:9092 --partitions 2

Created topic demo_2.


In [7]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --bootstrap-server localhost:9092 --topic demo_2 --describe

Topic: demo_2	PartitionCount: 2	ReplicationFactor: 1	Configs: 
	Topic: demo_2	Partition: 0	Leader: 1	Replicas: 1	Isr: 1
	Topic: demo_2	Partition: 1	Leader: 1	Replicas: 1	Isr: 1


### Send message into first topic

In [53]:
!echo "1,1,alice" | ./kafka_2.13-3.1.0/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic demo_1 --property "parse.key=true" --property "key.separator=,"


In [13]:
!echo "2,bob" | ./kafka_2.13-3.1.0/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic demo_1 --property "parse.key=true" --property "key.separator=,"
!echo "3,charlie" | ./kafka_2.13-3.1.0/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic demo_1 --property "parse.key=true" --property "key.separator=,"

Read messages

In [14]:
!./kafka_2.13-3.1.0/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic demo_1 --from-beginning

bob
alice
charlie
^C
Processed a total of 3 messages


### Send message into 2nd topic

In [15]:
!echo "1,alice information updated" | ./kafka_2.13-3.1.0/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic demo_2 --property "parse.key=true" --property "key.separator=,"

Read messages

In [16]:
!./kafka_2.13-3.1.0/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic demo_2 --from-beginning

alice information updated
^C
Processed a total of 1 messages


### Perform a Join

Using module ksql to achieve a SQL interface over Kafka Stream

In [17]:
!pip install ksql

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.8/269.8 KB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 KB 16.7 MB/s eta 0:00:00
  Created wheel for ksql: filename=ksql-0.10.2-py3-none-any.whl size=12800 sha256=53a269d4b1055424ff31dbc90cf7eae959f1433eb34c13cd12e971132d4ea4c2
  Stored in directory: /home/desenfirman/.cache/pip/wheels/0f/40/c1/9fbdca865566449b00a3a6e0427a6cf08ce5a359dea6c684ff
Successfully built ksql


Setup the KSQL client

In [22]:
import logging
from ksql import KSQLAPI
logging.basicConfig(level=logging.DEBUG)
client = KSQLAPI('http://localhost:8088')

client.ksql('show topics')

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8088
DEBUG:urllib3.connectionpool:http://localhost:8088 "GET /info HTTP/1.1" 200 None
DEBUG:root:KSQL generated: show topics


[{'@type': 'kafka_topics',
  'statementText': 'show topics;',
  'topics': [{'name': '_confluent-command', 'replicaInfo': [1]},
   {'name': '_confluent-controlcenter-5-4-0-1-actual-group-consumption-rekey',
    'replicaInfo': [1]},
   {'name': '_confluent-controlcenter-5-4-0-1-aggregate-topic-partition-store-changelog',
    'replicaInfo': [1]},
   {'name': '_confluent-controlcenter-5-4-0-1-aggregatedTopicPartitionTableWindows-ONE_MINUTE-changelog',
    'replicaInfo': [1]},
   {'name': '_confluent-controlcenter-5-4-0-1-aggregatedTopicPartitionTableWindows-ONE_MINUTE-repartition',
    'replicaInfo': [1]},
   {'name': '_confluent-controlcenter-5-4-0-1-aggregatedTopicPartitionTableWindows-THREE_HOURS-changelog',
    'replicaInfo': [1]},
   {'name': '_confluent-controlcenter-5-4-0-1-aggregatedTopicPartitionTableWindows-THREE_HOURS-repartition',
    'replicaInfo': [1]},
   {'name': '_confluent-controlcenter-5-4-0-1-AlertHistoryStore-changelog',
    'replicaInfo': [1]},
   {'name': '_confluent

In [23]:
client.ksql('show streams')

DEBUG:root:KSQL generated: show streams


[{'@type': 'streams',
  'statementText': 'show streams;',
  'streams': [{'type': 'STREAM',
    'name': 'KSQL_PROCESSING_LOG',
    'topic': 'default_ksql_processing_log',
    'format': 'JSON'}],
  'warnings': []}]

In [24]:
client.ksql('show tables')

DEBUG:root:KSQL generated: show tables


[{'@type': 'tables',
  'statementText': 'show tables;',
  'tables': [],
  'warnings': []}]

Things that I learn here. Streams are different with topics. Stream use source from topic. So, I need to create a stream using base topics that I created before

In [57]:
client.ksql("""
    CREATE STREAM demo_1_stream (
        ROWKEY STRING KEY,
        name STRING
    ) WITH (
        KAFKA_TOPIC = 'demo_1',
        VALUE_FORMAT = 'DELIMITED'
    )
""")

DEBUG:root:KSQL generated: 
    CREATE STREAM demo_1_stream (
        ROWKEY STRING KEY,
        name STRING
    ) WITH (
        KAFKA_TOPIC = 'demo_1',
        VALUE_FORMAT = 'DELIMITED'
    )



[{'@type': 'currentStatus',
  'statementText': "CREATE STREAM demo_1_stream (\n        ROWKEY STRING KEY,\n        name STRING\n    ) WITH (\n        KAFKA_TOPIC = 'demo_1',\n        VALUE_FORMAT = 'DELIMITED'\n    )\n;",
  'commandId': 'stream/`DEMO_1_STREAM`/create',
  'commandStatus': {'status': 'SUCCESS', 'message': 'Stream created'},
  'commandSequenceNumber': 3,
  'warnings': []}]

In [70]:
client.ksql("""
    CREATE STREAM demo_2_stream (
        ROWKEY STRING KEY,
        name STRING
    ) WITH (
        KAFKA_TOPIC = 'demo_2',
        VALUE_FORMAT = 'DELIMITED'
    )
""")

DEBUG:root:KSQL generated: 
    CREATE STREAM demo_2_stream (
        ROWKEY STRING KEY,
        name STRING
    ) WITH (
        KAFKA_TOPIC = 'demo_2',
        VALUE_FORMAT = 'DELIMITED'
    )



[{'@type': 'currentStatus',
  'statementText': "CREATE STREAM demo_2_stream (\n        ROWKEY STRING KEY,\n        name STRING\n    ) WITH (\n        KAFKA_TOPIC = 'demo_2',\n        VALUE_FORMAT = 'DELIMITED'\n    )\n;",
  'commandId': 'stream/`DEMO_2_STREAM`/create',
  'commandStatus': {'status': 'SUCCESS', 'message': 'Stream created'},
  'commandSequenceNumber': 6,
  'warnings': []}]

Test push query
https://docs.conduktor.io/features/ksqldb/how-to-query-with-ksqldb

In [72]:
q = client.query("""
    SELECT *
    FROM demo_1_stream
    EMIT CHANGES
""", stream_properties={"ksql.streams.auto.offset.reset": "earliest"})
max_row = 3
i = 0
for item in q:
    if i > max_row:
        break 
    print(item)
    i += 1

DEBUG:root:KSQL generated: 
    SELECT *
    FROM demo_1_stream
    EMIT CHANGES



{"header":{"queryId":"none","schema":"`ROWTIME` BIGINT, `ROWKEY` STRING, `NAME` STRING"}}

{"row":{"columns":[1646153014996,"1","alice"]}}

{"row":{"columns":[1646153050443,"3","charlie"]}}

{"row":{"columns":[1646153047087,"2","bob"]}}



### Perform a Join

Join scenario: To get an updated name information from topic demo_2 within 1 hours. demo_1 act as source table and demo_2 act as incoming changes. Here I show the difference of changed name between two topics

In [75]:
q = client.query("""
    SELECT 
        A.ROWKEY,
        A.name AS old_name,
        B.name AS changes_name
    FROM demo_1_stream A
        LEFT JOIN demo_2_stream B WITHIN 1 HOURS ON A.ROWKEY = B.ROWKEY
    EMIT CHANGES
""", stream_properties={"ksql.streams.auto.offset.reset": "earliest"})
max_row = 4
i = 0
for item in q:
    if i > max_row:
        break 
    print(item)
    i += 1

DEBUG:root:KSQL generated: 
    SELECT 
        A.ROWKEY,
        A.name AS demo_1_name,
        B.name AS demo_2_name
    FROM demo_1_stream A
        LEFT JOIN demo_2_stream B WITHIN 1 HOURS ON A.ROWKEY = B.ROWKEY
    EMIT CHANGES



{"header":{"queryId":"none","schema":"`A_ROWKEY` STRING, `DEMO_1_NAME` STRING, `DEMO_2_NAME` STRING"}}

{"row":{"columns":["2","bob",null]}}

{"row":{"columns":["1","alice",null]}}

{"row":{"columns":["3","charlie",null]}}

{"row":{"columns":["1","alice","alice information updated"]}}



Done :)